In [5]:
%matplotlib inline
import cv2
import numpy as np
from matplotlib import pyplot as plt
import imutils
from imutils import perspective
import math
import time

In [6]:
resW = 1920
resH = 1080

cap = cv2.VideoCapture(1)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, resW)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, resH)
#cap.set(cv2.CAP_PROP_FPS, 20)

font = cv2.FONT_HERSHEY_TRIPLEX
clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
detected = 0
capture = 0
save = 0
sizeOut = 512
rotate = 0

textString = ''
colorFPS = (255,255,0)
colorRect = (0, 255, 0)

#region extraction parameters
sizeRect = (21, 7)
sizeBlur = (9, 9)
iterErode = 10
iterDilate = 25
threshGrad = 200

boxExtend = 20
sortBar = 5

minLineLength = 150
maxLineGap = 10

#image exhancement parameters
threshBar = 125

#command parameters
cmdAngle = 5
cmdErode = 5

In [7]:
while(True):
    tick1 = cv2.getTickCount()
    (grabbed, img) = cap.read()
    
    if not grabbed:
        break
    
    if rotate != 0:
        img = imutils.rotate(img, angle=rotate)
    imgOrig = img.copy()
    detected = 0
    
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    imgGray = clahe.apply(imgGray)
    
    gradX = cv2.Sobel(imgGray, ddepth = cv2.CV_32F, dx = 1, dy = 0, ksize = -1)
    gradY = cv2.Sobel(imgGray, ddepth = cv2.CV_32F, dx = 0, dy = 1, ksize = -1)
    gradient = cv2.subtract(gradX, gradY)
    gradient = cv2.convertScaleAbs(gradient)
        
    blurred = cv2.blur(gradient, sizeBlur)
    (_, thresh) = cv2.threshold(blurred, threshGrad, 255, cv2.THRESH_BINARY)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, sizeRect)
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    closed = cv2.erode(closed, None, iterations = iterErode)
    closed = cv2.dilate(closed, None, iterations = iterDilate)
    (_, contours, _) = cv2.findContours(closed.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) >= 1:
        detected = 1
        if len(contours) >= sortBar:
            contours = sorted(contours, key = cv2.contourArea, reverse = True)[:sortBar]
        else:
            contours = sorted(contours, key = cv2.contourArea, reverse = True)[:len(contours)]
    
    countContour = 0
    for c in contours:
        rect = cv2.minAreaRect(c)
        box = np.int0(cv2.boxPoints(rect))
        box[0][0] -= boxExtend
        box[0][1] += boxExtend
        box[1][0] -= boxExtend
        box[1][1] -= boxExtend
        box[2][0] += boxExtend
        box[2][1] -= boxExtend
        box[3][0] += boxExtend
        box[3][1] += boxExtend
    
        imgCont = cv2.drawContours(img, [box], -1, colorRect, 3)
        
        if capture is 1:
            imgWarp = perspective.four_point_transform(imgOrig, box.reshape(4, 2))
            imgWarpGray = cv2.cvtColor(imgWarp, cv2.COLOR_BGR2GRAY)
            #imgWarpBlur = cv2.bilateralFilter(imgWarpGray, 5, 75, 75)
            #(retWarp, thWarp) = cv2.threshold(imgWarpGray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
            
            #edges = imutils.auto_canny(imgWarpGray)
            
            #(retWarp, threshWarp) = cv2.threshold(edges, 127, 255, cv2.THRESH_BINARY_INV)
            #(_, contoursWarp, _) = cv2.findContours(imgWarpGray.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            #imgWarpGray = cv2.drawContours(imgWarpGray, [contoursWarp], -1, colorRect, cv2.FILLED)

            #threshWarp = cv2.cvtColor(threshWarp, cv2.COLOR_GRAY2RGB)
            
            #for cw in contoursWarp:
            #    peri = cv2.arcLength(cw, True)
            #    approx = cv2.approxPolyDP(cw, 0.02 * peri, True)
            #    imgWarpGray = cv2.drawContours(imgWarpGray, [approx], -1, colorRect, cv2.FILLED)

            #lines = cv2.HoughLinesP(edges, 1, np.pi/180, 30, minLineLength, maxLineGap)
            #lines1 = lines[:,0,:]
            #for x1,y1,x2,y2 in lines1[:]: 
            #    cv2.line(img,(x1,y1),(x2,y2),(255,0,0),1)
 
            #imgWarpGray = cv2.fastNlMeansDenoising(imgWarpGray, None, 10, 10, 7)
            #imgWarpGray = cv2.equalizeHist(imgWarpGray)
            imgWarpGray = clahe.apply(imgWarpGray)
            #imgWarpGray = cv2.GaussianBlur(imgWarpGray, (5,5), 0)  
            (retWarp, thWarp) = cv2.threshold(imgWarpGray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
            cv2.imshow("Warp " + str(countContour), thWarp)
            if save is 1:
                fileName = "../imageCapture/" + time.strftime("%Y-%m-%d_%H%M%S-", time.localtime()) + str(countContour) + '.png'
                cv2.imwrite(fileName, thWarp, [int(cv2.IMWRITE_PNG_COMPRESSION), 9])
            
        countContour += 1
    
    if capture is 1 and save is 1:
        save = 0
    
    if detected is 1:
        tick2 = cv2.getTickCount()
        tick = math.floor( ((tick2 - tick1) * 1000) / cv2.getTickFrequency() )
        
        if rotate != 0:
            imgCont = imutils.rotate(imgCont, angle=(360 - rotate))
            #closed = imutils.rotate(closed, angle=(360 - rotate))
        
        imgCont = cv2.copyMakeBorder(imgCont, 0, 80, 0, 0, cv2.BORDER_CONSTANT, value=(0, 0, 0))
        textString = str(tick) + " ms, angle: " + str(rotate)
        cv2.putText(imgCont, textString, (10, resH+30), font, 1, colorFPS, 2, cv2.LINE_AA)
        textString = "Erode: " + str(iterErode) + " Dilate: " + str(iterDilate)
        cv2.putText(imgCont, textString, (10, resH+70), font, 1, colorFPS, 2, cv2.LINE_AA)
        
        imgCont = imutils.resize(imgCont, height = sizeOut)
        #closed = imutils.resize(closed, height = sizeOut)
        cv2.imshow("Output", imgCont)
        #cv2.imshow("Closed", closed)
    
    capture = 0
    getKey = cv2.waitKey(10) & 0xFF
    if getKey is ord('c'):
        capture = 1
    elif getKey is ord('v'):
        save = 1         
    elif getKey is ord('r'):
        if rotate >= 360:
            rotate = 0
        else:
            rotate += cmdAngle
    elif getKey is ord('t'):
        if rotate <= 0:
            rotate = 355
        else:
            rotate -= cmdAngle
    elif getKey is ord('z'):
        iterErode += cmdErode
        iterDilate += cmdErode
    elif getKey is ord('x'):
        iterErode -= cmdErode
        iterDilate -= cmdErode
    elif getKey is ord('a'):
        iterDilate += cmdErode
    elif getKey is ord('s'):
        iterDilate -= cmdErode
    elif getKey is ord('w'):
        cv2.destroyAllWindows()
    elif getKey is ord('q'):
        break

In [8]:
cap.release()
cv2.destroyAllWindows()